In [1]:
import matplotlib.pyplot as plt
import cv2
import face_recognition
import os
import json
import numpy as np

# Use FaceEncodeDataSmall.json to find unique_trueName and model

In [4]:
# Restore Encode_data from the 'FaceEncodeData.json'

with open('FaceEncodeDataSmall.json', 'r') as file:
    read_data = json.load(file) 
    
dirList = read_data['dirList']
encodeList = read_data['encodeList']
NumList = read_data['NumList']

print("dirList", dirList[:5])
print("total:", len(encodeList), len(NumList), "skip:", len(dirList)-len(encodeList))
print("picture:", NumList[:5])
print("encodeList[0]:", encodeList[0][:5])

dirList ['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg']
total: 5308 5308 skip: 1647
picture: [0, 1, 2, 3, 4]
encodeList[0]: [0.4156585708260536, 0.5184201896190643, 0.5057678893208504, 0.47036184556782246, 0.45827947184443474]


In [5]:
# Read true_data from the CSV file

import pandas as pd
CSVpath = "C:/Users/Mark Lam/My Drive (ironmanmail2@gmail.com)/Share/University_daily/Year4/Sem2/ECE50024_MachineLearning/MiniProject/data/train.csv"
CSV = pd.read_csv(CSVpath)
CSV_NumList = CSV.iloc[:, 0].tolist()  # Convert column 1 to a list
CSV_NameList = CSV.iloc[:, 2].tolist()  # Convert column 3 to a list
print("CSV_NumList", CSV_NumList[:5])
print("CSV_NameList", CSV_NameList[:5])
print("total", len(CSV_NumList))

CSV_NumList [0, 1, 2, 3, 4]
CSV_NameList ['Audrey Tautou', 'Adam Sandler', 'Anna Paquin', 'Ava Gardner', 'Amy Adams']
total 69540


In [6]:
# Extract true data from CSV_NameList

extra_trueName = []
for Num in NumList:
    if Num == CSV_NumList[Num]:
        extra_trueName.append(CSV_NameList[Num]) 
    else:
        print("ERROR file name")

print(extra_trueName[:5])
print(len(NumList), len(extra_trueName))

['Audrey Tautou', 'Adam Sandler', 'Anna Paquin', 'Ava Gardner', 'Amy Adams']
5308 5308


In [7]:
# Preparing model training

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

extra_trueName = np.array(extra_trueName)
_, idx = np.unique(extra_trueName, return_index=True)
num_classes = len(idx)

sorted_idx = np.sort(idx)
unique_trueName = extra_trueName[sorted_idx]

print(extra_trueName)
print("")
print(unique_trueName)
print(num_classes)

y_train = [[0 for _ in range(len(unique_trueName))] for _ in range(len(extra_trueName))]

# Now fill y_train based on the condition
for ans_index, ans_name in enumerate(extra_trueName):
    for cat_index, cat_name in enumerate(unique_trueName):
        if cat_name == ans_name:
            y_train[ans_index][cat_index] = 1

y_train = np.array(y_train)
print("")
print(y_train[11])
print(unique_trueName[np.argmax(y_train[11])])
print(y_train.shape)

input_shape = (128, 1)

encodeList = np.array(encodeList)
x_train = np.expand_dims(encodeList, -1)
print("x_train", x_train.shape)



['Audrey Tautou' 'Adam Sandler' 'Anna Paquin' ... 'Aaron Judge'
 'Bernie Mac' 'Ashley Judd']

['Audrey Tautou' 'Adam Sandler' 'Anna Paquin' 'Ava Gardner' 'Amy Adams'
 'Angelina Jolie' 'Bill Paxton' 'Alan Alda' 'Abigail Breslin'
 'Alicia Vikander' 'Albert Finney' 'Beyonce Knowles' 'Andrew Lincoln'
 'Armin Mueller-Stahl' 'Amy Ryan' 'Adrien Brody' 'Analeigh Tipton'
 'Amanda Bynes' 'Anne Bancroft' 'Albert Brooks' 'Barbra Streisand'
 'Andy Samberg' 'Annette Bening' 'Anthony Perkins' 'Alice Eve'
 'Berenice Bejo' 'AnnaSophia Robb' 'Aaron Paul' 'Alan Rickman'
 'Anjelica Huston' 'Betty White' 'Adele' 'Art Carney' 'Adriana Barraza'
 'Billy Bob Thornton' 'Alex Pettyfer' 'Barabara Palvin'
 'Alessandra Ambrosio' 'Andreea Diaconu' 'Benedict Cumberbatch'
 'Ben Kingsley' 'Anna Friel' 'Alx James' 'Andrew Garfield'
 'Antonio Banderas' 'Anna Kendrick' 'Amanda Seyfried' 'Alan Arkin'
 'Ali Larter' 'Bill Daley' 'Ben Johnson' 'Amber Heard' 'Anderson Cooper'
 'Al Roker' 'Barry Pepper' 'Bernie Mac' 'Alec Baldw

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
input_shape = (128, 1)
num_classes = 100
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Conv1D(32, kernel_size=1, activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=1, activation="relu"), #128 or 192
    layers.MaxPooling1D(pool_size=2),
    #layers.Conv1D(64, kernel_size=1, activation="relu"),
    #layers.MaxPooling1D(pool_size=2),
    #layers.Conv1D(128, kernel_size=1, activation="relu"),
    #layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(num_classes, activation="softmax"),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 128, 32)           64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 64, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 64, 128)           4224      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 32, 128)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 100)               4

# Real test by Bigmodel Method2

In [73]:
# Encoding all the 6955 images in small train
'''
path = "C:/Users/Mark Lam/My Drive (ironmanmail2@gmail.com)/Share/University_daily/Year4/Sem2/ECE50024_MachineLearning/MiniProject/data/test"
dirList = os.listdir(path)
dirList = sorted(dirList, key=lambda x: int(x.split('.')[0]))

skip = 0
print(dirList[:5])
print("len", len(dirList))


for cl in dirList:
    curImg = face_recognition.load_image_file(f'{path}/{cl}')
    curName = int((os.path.splitext(cl)[0]))
    if curName%10 == 0:
        print("NowPic:", curName/len(dirList)*100, "%...skip:", skip)

    MaxLeg = max(curImg.shape)
    curImg = cv2.resize(curImg, (0,0), None, min(1,1000/MaxLeg), min(1,1000/MaxLeg))
    curImg = cv2.cvtColor(curImg, cv2.COLOR_BGR2RGB)
    faceLocTemp = face_recognition.face_locations(curImg)

    encode = []
    faceLoc = []
    if len(faceLocTemp) != 0:  #skip the no face
        for i in range(len(faceLocTemp)):
            faceLoc.append(faceLocTemp[i])
            encode.extend((face_recognition.face_encodings(curImg)[i] + 1)/2)  # scaled to 0~1
        #encode = face_recognition.face_encodings(curImg)[0]
        #encodeList.append(encode.tolist())
        #NumList.append(curName)
        save_data = {
            'dirList': dirList,
            'encodeList': encode,
            'NumList': curName,
        }   
        if curName == 0:
            with open('BigtestWithSmallM2.json', 'w') as file:
                json.dump(save_data, file)
        else:
            with open('BigtestWithSmallM2.json', 'a') as file:
                file.write('\n' + json.dumps(save_data))
    else:
        skip += 1
'''

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg']
len 4977
NowPic: 0.0 %...skip: 0
NowPic: 0.20092425155716293 %...skip: 0
NowPic: 0.40184850311432585 %...skip: 1
NowPic: 0.6027727546714888 %...skip: 2
NowPic: 0.8036970062286517 %...skip: 2
NowPic: 1.0046212577858147 %...skip: 4
NowPic: 1.2055455093429777 %...skip: 4
NowPic: 1.4064697609001406 %...skip: 8
NowPic: 1.6073940124573034 %...skip: 9
NowPic: 1.8083182640144666 %...skip: 9


C:\Users\Mark Lam\.conda\envs\tf\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


NowPic: 2.0092425155716294 %...skip: 9
NowPic: 2.210166767128792 %...skip: 10
NowPic: 2.4110910186859553 %...skip: 13
NowPic: 2.6120152702431185 %...skip: 13
NowPic: 2.8129395218002813 %...skip: 15
NowPic: 3.0138637733574445 %...skip: 16
NowPic: 3.214788024914607 %...skip: 17
NowPic: 3.4157122764717704 %...skip: 19
NowPic: 3.616636528028933 %...skip: 19
NowPic: 3.817560779586096 %...skip: 20
NowPic: 4.018485031143259 %...skip: 21
NowPic: 4.219409282700422 %...skip: 23
NowPic: 4.420333534257584 %...skip: 26
NowPic: 4.621257785814748 %...skip: 27
NowPic: 4.822182037371911 %...skip: 29
NowPic: 5.023106288929074 %...skip: 32
NowPic: 5.224030540486237 %...skip: 34
NowPic: 5.424954792043399 %...skip: 34
NowPic: 5.625879043600563 %...skip: 35
NowPic: 5.826803295157726 %...skip: 38
NowPic: 6.027727546714889 %...skip: 39
NowPic: 6.228651798272051 %...skip: 40
NowPic: 6.429576049829214 %...skip: 42
NowPic: 6.630500301386377 %...skip: 44
NowPic: 6.831424552943541 %...skip: 44
NowPic: 7.0323488045

In [9]:
with open('BigtestWithSmallM2.json', 'r') as file:
    read_data = [json.loads(line) for line in file]


dirList = read_data[0]['dirList']
dirList = np.array(dirList)
encodeList = [item['encodeList'] for item in read_data]
#encodeList = np.array(encodeList)
    
    
NumList = [item['NumList'] for item in read_data]
NumList = np.array(NumList)
print(encodeList[0][:5])
print(NumList)
print("")
print(dirList.shape)
print(len(encodeList))
print(NumList.shape)

[0.44127265736460686, 0.550319004803896, 0.5496292933821678, 0.486734701320529, 0.40213268995285034]
[   0    1    2 ... 4974 4975 4976]

(4977,)
4504
(4504,)


In [10]:
model.load_weights('my_modelBig3.h5')

In [11]:
idx = 9
for fileIndex, fileNum in enumerate(NumList):
    if idx == fileNum:
        print(len(encodeList[fileIndex]))
        yhatList = []
        for faceIndex in range(len(encodeList[fileIndex])//128):
            x_test = encodeList[fileIndex][faceIndex*128:(faceIndex+1)*128]
            x_test = np.array(x_test)
            x_test = x_test.reshape((1, 128, 1))
            print(x_test.shape)
            yhatList.append(model.predict(x_test[0:1], verbose=0))

        print("FaceAmount",len(yhatList))

        yhatMax = []
        yhatName = []
        for faceIndex in range(len(yhatList)):
            print(np.argmax(yhatList[faceIndex]))
            print(np.sort(yhatList[faceIndex][0])[::-1][:5])
            print(unique_trueName[np.argmax(yhatList[faceIndex][0])])
            yhatMax.append(np.sort(yhatList[faceIndex][0])[::-1][0])
            yhatName.append(unique_trueName[np.argmax(yhatList[faceIndex][0])])
        
        HighestFaceIndex = np.argmax(yhatMax)
        result = yhatName[HighestFaceIndex]
        print("")
        print("HighestFaceIndex", HighestFaceIndex)
        print("result:", result)


256
(1, 128, 1)
(1, 128, 1)
FaceAmount 2
35
[9.9194676e-01 2.6931281e-03 1.2687125e-03 8.8016683e-04 4.2649198e-04]
Alex Pettyfer
48
[0.28963983 0.18958558 0.1158065  0.05391093 0.05150501]
Ali Larter

HighestFaceIndex 0
result: Alex Pettyfer


In [12]:
import csv
fieldnames = ['Id', 'Category']
data = []
FileName = 'Result_bigmodelM2_5.csv' #1 2 3 4
#Result_bigmodelM2_ Using local_model 32 128 with model_checkpoint 83.3 82.4 114 'my_modelBig1.h5'
#Result_bigmodelM2_2 Using colab 32 128 early_stopper100 82 80 404
#Result_bigmodelM2_3 Using colab 32 192 early_stopper100 83 81 412
#Result_bigmodelM2_4 Using local_model 32 192 with model_checkpoint 83.7 82.5 248 'my_modelBig2.h5'
#Result_bigmodelM2_5 Using local_model 32 128 with model_checkpoint 82.6 82.4 720 'my_modelBig3.h5'

with open(FileName, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

TrueCounter = 0

print("dirList", len(dirList))

diff_indexList = []
diff_found = False
for i in range(len(dirList)):
    for fileIndex, fileNum in enumerate(NumList):
        if i == fileNum:
            #yhat = model.predict(x_test[fileIndex:(fileIndex+1)], verbose=0)
            #result = unique_trueName[np.argmax(yhat[0])]
            
            ################
            yhatList = []
            for faceIndex in range(len(encodeList[fileIndex])//128):
                x_test = encodeList[fileIndex][faceIndex*128:(faceIndex+1)*128]
                x_test = np.array(x_test)
                x_test = x_test.reshape((1, 128, 1))
                yhatList.append(model.predict(x_test[0:1], verbose=0))
    
            #print("FaceAmount",len(yhatList), "at", fileNum)

            yhatMax = []
            yhatName = []
            for faceIndex in range(len(yhatList)):
                yhatMax.append(np.sort(yhatList[faceIndex][0])[::-1][0])
                yhatName.append(unique_trueName[np.argmax(yhatList[faceIndex][0])])
            
            HighestFaceIndex = np.argmax(yhatMax)
            result = yhatName[HighestFaceIndex]
            ################
            
            TrueCounter += 1
            diff_found = False
            break
        else:
            result = "Ashley Greene"
            diff_found = True
    
            
    if diff_found == True:
        diff_indexList.append(i)
    
    data = [{"Id": i, "Category": result}]
    with open(FileName, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for row in data:
            writer.writerow(row)

    if i%100 == 0:
        print("NowPic:", i/len(dirList)*100, "%...")

print("check skip", len(dirList)-TrueCounter, len(diff_indexList))
print(diff_indexList)

dirList 4977
NowPic: 0.0 %...
NowPic: 2.0092425155716294 %...
NowPic: 4.018485031143259 %...
NowPic: 6.027727546714889 %...
NowPic: 8.036970062286517 %...
NowPic: 10.046212577858148 %...
NowPic: 12.055455093429778 %...
NowPic: 14.064697609001406 %...
NowPic: 16.073940124573035 %...
NowPic: 18.083182640144667 %...
NowPic: 20.092425155716295 %...
NowPic: 22.101667671287924 %...
NowPic: 24.110910186859556 %...
NowPic: 26.12015270243118 %...
NowPic: 28.129395218002813 %...
NowPic: 30.13863773357444 %...
NowPic: 32.14788024914607 %...
NowPic: 34.1571227647177 %...
NowPic: 36.166365280289334 %...
NowPic: 38.17560779586096 %...
NowPic: 40.18485031143259 %...
NowPic: 42.19409282700422 %...
NowPic: 44.20333534257585 %...
NowPic: 46.21257785814748 %...
NowPic: 48.22182037371911 %...
NowPic: 50.23106288929073 %...
NowPic: 52.24030540486236 %...
NowPic: 54.249547920434 %...
NowPic: 56.258790436005626 %...
NowPic: 58.268032951577254 %...
NowPic: 60.27727546714888 %...
NowPic: 62.28651798272051 %...

In [13]:
print(TrueCounter/len(dirList)*100)
# Never change 90.49628290134619

90.49628290134619


In [3]:
#check different %
import pandas as pd
File1 = "Result_bigmodelM2_.csv"
CSVpath = "C:/Users/Mark Lam/My Drive (ironmanmail2@gmail.com)/Share/University_daily/Year4/Sem2/ECE50024_MachineLearning/MiniProject/" + File1
CSV = pd.read_csv(CSVpath)
CSV_NameList = CSV.iloc[:, 1].tolist()  # Convert column 3 to a list
print(File1, CSV_NameList[0:5])

File2 = "Result_bigmodelM2_5.csv"
CSVpath = "C:/Users/Mark Lam/My Drive (ironmanmail2@gmail.com)/Share/University_daily/Year4/Sem2/ECE50024_MachineLearning/MiniProject/" + File2
CSV = pd.read_csv(CSVpath)
CSV_NameList2 = CSV.iloc[:, 1].tolist()  # Convert column 3 to a list
print(File2, CSV_NameList2[0:5])

diff_count = 0
CSV_IndexCounter = 0
diff_index = []
for CSV_Name, CSV_Name2 in zip(CSV_NameList, CSV_NameList2):
    if CSV_Name != CSV_Name2:
        diff_index.append(CSV_IndexCounter)
        diff_count += 1
    CSV_IndexCounter += 1

print(diff_index)
print("diff_count", diff_count)
print(diff_count/len(CSV_NameList)*100, "%")

Result_bigmodelM2_.csv ['Adele', 'Amy Ryan', 'Adrien Brody', 'Adriana Barraza', 'Albert Brooks']
Result_bigmodelM2_5.csv ['Anna Sui', 'Amy Ryan', 'Adrien Brody', 'Adriana Barraza', 'Albert Brooks']
[0, 26, 29, 40, 84, 109, 150, 154, 156, 161, 175, 206, 215, 227, 230, 256, 264, 278, 283, 287, 289, 316, 397, 416, 449, 466, 503, 516, 517, 536, 538, 540, 597, 598, 649, 656, 663, 675, 684, 705, 707, 719, 747, 748, 749, 755, 761, 768, 773, 778, 788, 792, 795, 842, 859, 866, 900, 906, 952, 956, 957, 960, 964, 969, 994, 999, 1006, 1042, 1062, 1093, 1097, 1102, 1114, 1139, 1148, 1183, 1185, 1195, 1197, 1226, 1235, 1258, 1265, 1273, 1277, 1278, 1285, 1295, 1297, 1301, 1314, 1318, 1336, 1347, 1356, 1361, 1366, 1395, 1414, 1432, 1433, 1451, 1483, 1486, 1496, 1516, 1525, 1526, 1529, 1539, 1540, 1576, 1592, 1609, 1616, 1632, 1635, 1651, 1655, 1681, 1708, 1709, 1728, 1754, 1755, 1757, 1759, 1764, 1784, 1793, 1801, 1808, 1829, 1888, 1890, 1910, 1917, 1918, 1938, 1962, 2016, 2027, 2041, 2055, 2098, 210